# Imports and definitions

In [4]:
import pandas as pd
file_path = "/project_ghent/confusion_lara/whole_dataset_noproblems_feedback.json"

df = pd.read_json(file_path)

In [5]:
df.loc[:, ["text", "speaker"]]

,text,speaker
0,"Hallo, ik ben Toffee Bot.Hoe heet jij?",robot
1,Robin.,user
2,Fijn jou te ontmoeten.Ik ben een robotkok en i...,robot
3,"Oei, daar weet ik niet zo heel veel van.",user
4,"Toffee is een soort taai, hard snoepgoed, mees...",robot
...,...,...
6175,"Nee, dat wist ik niet.",user
6176,Ik wilde je zo veel mogelijk laten bijleren ov...,robot
6177,"Ja, dat is geen enkel probleem.",user
6178,We zijn beland aan het einde van het recept.Ho...,robot


In [6]:
# Check if robot is always even and user is always uneven
for i, row in enumerate(df.loc[:, ["text", "speaker"]].iterrows()):
    if row[0] % 2 == 0 and row[1]["speaker"] != "robot":
        print("Even user")
    if row[0] % 2 == 1 and row[1]["speaker"] != "user":
        print("Uneven robot")

# Flan-t5-large

In [33]:
import sentencepiece
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [34]:
input_text = """Wat heb je vandaag gedaan?"""

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

input_text = """Can you tell me about your day?"""

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/opt/conda/lib/python3.11/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad>het hebben te hebben vanda
<pad> I was at the beach today.</s>


Dutch performance doesn't seem great

In [36]:
import os
import re

pattern = r'\[(.*?)\]'

cropped_root = "/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/"
collected_robot_response = False

TN, TP, FN, FP = 0, 0, 0, 0
amount = 0
for i, row in df.iterrows():
    if i % 2 == 0:
        video_endpath = str(row["video"]) + "_interaction_" + str(row["interaction"]) + "_utterance_" + str(row["utterance"]) + "_merged_cropped.mp4"
        label = "YES"
        video_path = cropped_root + "YES/" + video_endpath
        if not os.path.exists(video_path):
            label = "NO"
            video_path = cropped_root + "NO/" + video_endpath

    if not collected_robot_response:
        # Robot response
        if i % 2 == 0:
            robot_resp = row["text"]
            collected_robot_response = True
        else:
            print("Something went wrong")
    else:  
        # User response
        if i % 2 == 1:
            user_resp = row["text"]
            collected_robot_response = False
        else:
            print("Something went wrong")
        
        input_text = f"""Does the response of the user indicate that the robots made a mistake, provoked confusion or any other elements that caused a disturbance in the conversation? 
        \nRobot: {robot_resp} \nUser: {user_resp}
        \nAnswer only with '[YES]', '[NO]' or '[MAYBE]' in case of no sufficient information"""
        
        # input_text = f"""Duidt de reactie van de gebruiker erop dat de robots een fout hebben gemaakt, verwarring hebben veroorzaakt of andere elementen die verstoring in het gesprek hebben veroorzaakt?
        #  \nRobot: {robot_resp} \nGebruiker: {user_resp}
        #  \nAntwoord alleen met '[JA]', '[NEE]' of '[MISSCHIEN]' in geval van onvoldoende informatie"""
        
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

        outputs = model.generate(input_ids)
        response = tokenizer.decode(outputs[0])
        classification = re.findall(pattern, response)[0]
        if classification == "NO" and label == "NO":
            TN += 1
        elif classification == "NO" and label == "YES":
            FN += 1
        elif classification == "YES" and label == "YES":
            TP += 1
            print(f"{robot_resp} | {user_resp}")
        elif classification == "YES" and label == "NO":
            FP += 1
            print(f"{robot_resp} | {user_resp}")
            
        amount += 1

print("----------------")
print("TN:", TN)
print("FN:", FN)
print("TP:", TP)
print("FP:", FP)
print("Accuracy:", (TP + TN) / (TP + TN + FP + FN))
print("F1:", (2*TP) / (2*TP + FP + FN))

/opt/conda/lib/python3.11/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


 Wanneer tijdens het koken de kruiden hun smaak hebben afgegeven, kan je de oregano takjes verwijderen.Na het pureren zet je de soep terug op het vuur.Voeg dan de orzo en een halve liter water toe.Kook ongeveer 10 minuten totdat de orzo gaar is.Nu is de soep klaar om geserveerd te worden.Dat ging snel, vond je niet? | Ja, ging snel.
Schep de spinazie nu voorzichtig door de risotto tot de blaadjes zijn gezakt.Neem dan de pan van het vuur en meng twee derde van een zakje parmezaanse kaas met de roomboter.Klinkt dat goed? | Ja, dat klinkt oké.Dat klinkt goed.
Klopt.Brussel is de hoofdstad van België. | Dat had ik niet gevraagd, maar dat is inderdaad zo.Brussel is inderdaad de hoofdstad van België.
Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | 
 Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | 
Ik wilde je zoveel mogelijk laten bijleren over het recept, maar ik zie nu dat sommige informatie overbodig was. | Ja, sommige informatie was een beetj

# GPT-3.5-turbo

In [5]:
from openai import OpenAI
api_key = "Ah, too bad. Not today!"
client = OpenAI(api_key = api_key)

In [28]:
def prompt_gpt(robot_resp, user_resp):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {
          "role": "system",
          "content": """Does the response of the user indicate that the robots made a mistake, provoked confusion or any other elements that caused a disturbance in the conversation? 
          Answer only with '[YES]', '[NO]' or '[MAYBE]' in case of no sufficient information"""
          # "content": """Heeft de reactie van de gebruiker aangegeven dat de robots een fout hebben gemaakt, verwarring hebben veroorzaakt of andere elementen die verstoring in het gesprek hebben veroorzaakt? 
            # Antwoord alleen met '[JA]', '[NEE]' of '[MISSCHIEN]' in geval van onvoldoende informatie."""
        },
        {
          "role": "user",
          "content": f"Robot: {robot_resp} \nUser: {user_resp}"
        },  
      ],
      temperature=0.7,
      max_tokens=64,
      top_p=1
    )
    return response.choices[0].message.content


In [19]:
robot_resp = "Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen."
user_resp = "Excuses aanvaard."

answer = prompt_gpt(robot_resp, user_resp)
print(answer)

[NEE]


In [29]:
import os
import re

pattern = r'\[(.*?)\]'

cropped_root = "/project_ghent/Master-Thesis/no_robot_audio/labelled/CROPPED/"
collected_robot_response = False

TN, TP, FN, FP = 0, 0, 0, 0
amount = 0
for i, row in df.iterrows():
    if i % 2 == 0:
        video_endpath = str(row["video"]) + "_interaction_" + str(row["interaction"]) + "_utterance_" + str(row["utterance"]) + "_merged_cropped.mp4"
        label = "YES"
        video_path = cropped_root + "YES/" + video_endpath
        if not os.path.exists(video_path):
            label = "NO"
            video_path = cropped_root + "NO/" + video_endpath

    if not collected_robot_response:
        # Robot response
        if i % 2 == 0:
            robot_resp = row["text"]
            collected_robot_response = True
        else:
            print("Something went wrong")
    else:  
        # User response
        if i % 2 == 1:
            user_resp = row["text"]
            collected_robot_response = False
        else:
            print("Something went wrong")
        
        response = prompt_gpt(robot_resp, user_resp)
        classification = re.findall(pattern, response)[0]
        if classification == "NO" and label == "NO":
            TN += 1
        elif classification == "NO" and label == "YES":
            FN += 1
        elif classification == "YES" and label == "YES":
            TP += 1
            print(f"{robot_resp} | {user_resp}")
        elif classification == "YES" and label == "NO":
            FP += 1
            print(f"{robot_resp} | {user_resp}")
        
        amount += 1

print("----------------")
print("TN:", TN)
print("FN:", FN)
print("TP:", TP)
print("FP:", FP)
print("Accuracy:", (TP + TN) / (TP + TN + FP + FN))
print("F1:", (2*TP) / (2*TP + FP + FN))

 Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | 
Vroeg je dat niet?Ah, je foutje.Maar dat komt helemaal goed vanaf nu. | 
 Het ziet er naar uit dat je niet vroeg om over het weer te praten.Ik zal het rechtzetten, beloofd. | 
Ai, soms weet ik niet wanneer te stoppen. | Schinnerig.
 Ik kan je garanderen dat met de geitenkaas en zoete aardappel de spinazie zou smaken als nooit tevoren.Zorg dat je zachte geitenkaas gebruikt en voeg om het gerecht af de werken nog een handje gemalen walnootjes toe.Geitenkaas heeft een erg uitgesproken smaak.Deze is daarom niet te vergelijken met kaas die van koeienmelk gemaakt wordt. De specifieke smaak komt van een onderstofje dat in geitenkaas zit, namelijk caprinezuur.Caprine verwijst naar cabra, dat betekent geit, omdat het melkvet van geitenmelk voor 15% uit vetsure esters van capronzuur, caprilinezuur en caprinezuur bestaat.Is dat duidelijk? | Nee, eigenlijk niet.
 Ah, daar vroeg je niet om.Zo dom van mij, sorry. | 
Wauw,

# Compare amount of sentences classified as YES

In [37]:
text = """Wanneer tijdens het koken de kruiden hun smaak hebben afgegeven, kan je de oregano takjes verwijderen.Na het pureren zet je de soep terug op het vuur.Voeg dan de orzo en een halve liter water toe.Kook ongeveer 10 minuten totdat de orzo gaar is.Nu is de soep klaar om geserveerd te worden.Dat ging snel, vond je niet? | Ja, ging snel.
Schep de spinazie nu voorzichtig door de risotto tot de blaadjes zijn gezakt.Neem dan de pan van het vuur en meng twee derde van een zakje parmezaanse kaas met de roomboter.Klinkt dat goed? | Ja, dat klinkt oké.Dat klinkt goed.
Klopt.Brussel is de hoofdstad van België. | Dat had ik niet gevraagd, maar dat is inderdaad zo.Brussel is inderdaad de hoofdstad van België.
Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | 
 Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | 
Ik wilde je zoveel mogelijk laten bijleren over het recept, maar ik zie nu dat sommige informatie overbodig was. | Ja, sommige informatie was een beetje overbodig.
Wanneer tijdens het koken de kruiden hun smaak hebben afgegeven, kan je de oregano takjes verwijderen.Na het pureren zet je de soep terug op het vuur.Voeg dan de orzo in een halve liter water toe.Kook ongeveer 10 minuten totdat de orzo gaar is.Nu is de soep klaar om geserveerd te worden. Dat ging snel, vond je niet? | Ja, dat ging inderdaad snel.
 Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | Oké.Excuses aanvaard.
 Als eerste neem je een kookpan en zet je hem op een laag vuur.Doe de boter in de pan en laat deze smelten.Voeg nu de gecondenseerde melk toe en roer dit erdoor.Ben jij nog mee? | Ja, ik ben nog mee.
 Oh, ik dacht dat je al klaar was met praten. | Ik was klaar met praten.
De lekkerste alternatieven voor kersttomaten in dit recept zijn zongdroogde tomaten of rode paprika. |  Ja, dat klopt, maar ik was eigenlijk nog aan het nadenken.En je hebt me onderbroken.
 Je kan de buree op smaak brengen met peper en zout.Dan kunnen we nu de spinazie toevoegen.Schep de spinazie erdoor tot deze iets begint te verminderen in omvang.Eet je graag spinazie. | Ja, heel graag.
Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | 
 Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | 
 De stamppot is nu klaar.De meeste mensen eten hier ook graag worst bij.Je kan bijvoorbeeld chipolata worsten kopen.Die kan je ook vegetarisch vinden.Ben jij vegetarisch? | Ja, ik ben vegetariër.
Wanneer tijdens het koken de kruiden hun smaak hebben afgegeven, kan je de oregano takjes verwijderen.Na het pureren zet je de soep terug op het vuur.Voeg dan de orzo en een halve liter water toe.Kook ongeveer 10 minuten totdat de orzo gaar is.Nu is de soep klaar om geserveerd te worden. Dat ging snel, vond je niet? | Ja, dat ging heel snel.
 Als eerste stap doe je de roomboter in de mok en smelt je de boter voor 30 seconden in de microgolfoven op 800 Watt.Heb je thuis een mok die in de microgolfoven mag? | Ja, we hebben er verschillende.
 Hallo, ik ben Stamppot.Wat is jouw naam? | Mijn naam is Roelands.Wat is jouw naam?
Je kan de puree op smaak brengen met peper en zout.Dan kunnen we nu de spinazie toevoegen.Schep de spinazie erdoor tot deze iets begint te verminderen in omvang.Eet je graag spinazie. |  Ah ja, heel graag.
Oh, ik dacht dat je al klaar was met praten. |  Ik was ook klaar met praten.
Oh, ik dacht dat je al klaar was met praten. | Ik was al klaar met praten.
 Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | 
Ik zie dat ik je onderbroken heb.Er is nog wat werk aan de timing van mijn antwoorden. | Ja, je had een vraag gesteld en ik heb niet echt kans gekregen om te antwoorden.
Oh, ik dacht dat je al klaar was met praten. |  Ik was klaar met praten.
Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | Oké.
Oh, ik dacht dat je al klaar was met praten. | Ja, ik was al klaar met praten.
Verdeel nu de risotto over de borden en schep de tomaatjes of paprika erover.We zijn al bijna aan het einde beland. Doe nu de volgende stap van het recept en dan ben je klaar.Is dat duidelijk? | Wat is de volgende stap van het recept?
Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | 
Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | 
Oh, ik dacht dat je al klaar was met praten. | Ik was klaar met praten.
 Hallo, ik ben Soepot.Wie ben jij? | Ik ben Koen.Hallo, ik ben Soepot. Wie ben jij?Ik ben Koen.
Ik zie dat ik je onderbroken heb.Er is nog wat werk aan de timing van mijn antwoorden. | Dat klopt.
Froze yoghurt is echt het ideale zomerdessert.Het enige dat je nodig hebt is een pot Griekse yoghurt, twee eetlepels fijne suiker en één theelepel vanille-extract. Als eerste stap meng je de Griekse yoghurt, suiker en vanillegoed in een kom.Daarna heb je een platte schaal nodig om het mengsel over te verdelen.Kan je de ingrediënten nog eens herhalen voor me? | Griekse yoghurt, suiker en vanilleextract.
Ik zie dat ik je onderbroken heb.Er is nog wat werk aan de timing van mijn antwoorden. | Dat klopt.
 Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | Oké.
 Froze yoghurt is echt het ideale zomerdessert.Het enige dat je nodig hebt is een pot Griekse yoghurt, 2 eetlepels fijne suiker en 1 theelepel vanille extract.Als eerste stap meng je de Griekse yoghurt, suiker en vanille goed in een kom.Daarna heb je een platte schaal nodig om het mengsel over te verdelen.Kan je de ingrediënten nog eens herhalen voor me? | Griekse yoghurt, suiker en vanille extract?
Schep de spinazie nu voorzichtig door de risotto tot de blaadjes zijn gezakt. Neem dan de pan van het vuur en meng twee derde van een zakje parmezaanse kaas met de romboeter.Klinkt dat goed? | Ja, klinkt goed.
Ik zie dat ik je onderbroken heb.Ik wil me hiervoor veronschuldigen. | Excuses aanvaard.
 Schep de spinazie nu voorzichtig door de risotto tot de blaadjes zijn gezakt.Neem dan de pan van het vuur en meng 2 derde van een zakje parmezaanse kaas met de romboter.Klinkt dat goed? | Ja, dat klinkt lekker.
 Schep de spinazie nu voorzichtig door de risotto tot de blaadjes zijn gezakt.Neem dan de pan van het vuur en meng 2 derde van een zakje parmezaanse kaas met de roomboter.Klinkt dat goed? | Ja, klinkt goed."""

In [38]:
text.count("|")

40

In [39]:
text2 = """Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | 
Vroeg je dat niet?Ah, je foutje.Maar dat komt helemaal goed vanaf nu. | 
 Het ziet er naar uit dat je niet vroeg om over het weer te praten.Ik zal het rechtzetten, beloofd. | 
Ai, soms weet ik niet wanneer te stoppen. | Schinnerig.
 Ik kan je garanderen dat met de geitenkaas en zoete aardappel de spinazie zou smaken als nooit tevoren.Zorg dat je zachte geitenkaas gebruikt en voeg om het gerecht af de werken nog een handje gemalen walnootjes toe.Geitenkaas heeft een erg uitgesproken smaak.Deze is daarom niet te vergelijken met kaas die van koeienmelk gemaakt wordt. De specifieke smaak komt van een onderstofje dat in geitenkaas zit, namelijk caprinezuur.Caprine verwijst naar cabra, dat betekent geit, omdat het melkvet van geitenmelk voor 15% uit vetsure esters van capronzuur, caprilinezuur en caprinezuur bestaat.Is dat duidelijk? | Nee, eigenlijk niet.
 Ah, daar vroeg je niet om.Zo dom van mij, sorry. | 
Wauw, dat was overdreven veel info.Sorry. | 
 Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | 
Je moet de takjes verwijderen als deel van het recept.Is dat duidelijk? | Dat is geen reden, maar ja, oké.Soms wat doen.
Vroeg je dat niet?Ui, foutje.Maar dat komt helemaal goed vanaf nu. | Ja, ik ben akkoord.
Maak een vruchtensaus.Is dat duidelijk? | Nee, dat is geen extra info, dus dat weet ik niet zoveel.
Ik zie nu dat mijn antwoord niet relevant was.Het spijt mij zeer. | 
Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | 
Verdeel nu de risotto over de borden en schep de tomaatjes of paprika erover.We zijn al bijna aan het einde beland.Doe nu de volgende stap van het recept en dan ben je klaar.Is dat duidelijk? | Kunt u dat nog eens herhalen alsjeblieft?
Ik wilde je zoveel mogelijk laten bijleren over het recept, maar ik zie niet dat sommige informatie overbodig was. |  Ja, het was een beetje overbeidig aan het werk.
Ik zal je nu over de bereiding vertellen.Wat denk jij dat de eerste stap is? | De boter opwarmen.Met de bloem.
Ik zie dat ik te weinig informatie meegedeeld heb.Ik verzeker je dat dat de laatste keer was. | 
Wow, dat was overdreven veel info.Sorry. | 
Je moet de takjes verwijderen als deel van het recept.Is dat duidelijk? | Nee, dat is eigenlijk niet echt zijn reden.
Vroeg je dat niet? Ah, je foutje, maar dat komt helemaal goed vanaf nu. | 
 Ai, daar vroeg je niet om.Zo dom van mij, sorry. | Geen probleem.
Ik verzeker je dat de risotto die we hier gebruiken heel gemakkelijk bereid is.Terwijl de risotto rijst op het vuur staat, kan je intussen de kersttomaatjes en knoflook snijden.Zal dat lukken? | Ja, dan moet ik nog naar tanken.
Ik merkte dat ik meer details moest geven bij die laatste stap.Het is niet altijd zo gemakkelijk om in te schatten hoeveel informatie mensen juist willen. | 
Fijn jou te ontmoeten.Ik ben een robot kok en ik ga je vandaag leren hoe je een brownie in een mop maakt voor één persoon.De ingrediënten die je nodig hebt zijn 30 gram roomboter, 3 eitlepels tarwebloem, 3 eitlepels donkere wastaartsuiker, 1 zakje vanille suiker, 1 ei, Drie eetlepels halvolle melk en ten slotte één eetlepel cacaopoeder.Cacaopoeder is afkomstig van de cacaoboom, de vrucht van de cacaoboom.Om cacaopoeder te maken, wordt gebruikgemaakt van de cacaomassa van de cacaoboom.Onder hoge druk wordt de massa uitgeperst, zodat er cacaopoeder en cacaoboom te ontstaan.Ben je nog mee? | Ja, ik ben nog mee.Het was een beetje teveel, eigenlijk.
 Je moet de takjes verwijderen als deel van het recept.Is dat duidelijk? | Nee.
Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | Ja, ik vroeg mij af wat een schrikkeljaar te maken heeft met het recept.
Klopt, Brussel is de hoofdstad van België. | Dat heeft niks met het recept te maken.
Maak een vruchtensaus.Is dat duidelijk? | 
Oh, daar vroeg je niet om he?Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | 
Vroeg je dat niet?Ah, je foutje.Maar dat komt helemaal goed vanaf nu. | Oké, in orde.
De opties zijn eindeloos. Wat dacht je bijvoorbeeld van vers rood fruit zoals frambozen, aardbeien en bramen of koekjes zoals bakkiepotjes, stroopwafels, zandkoekjes, chocoladekoekjes en lange vingers, maar ook snoepjes zijn een optie zoals M&M's, sprinklers en knetters, eikrood, geroosterde zaden en noten zoals scheezaad, walnoot, hazelnoot, pecanoot en pistachenootjes.Kan je dat onthouden? | Nee, dat was teveel informatie.
Maak een vruchtensaus.Is dat duidelijk? |  Niet helemaal.
 Ik kan je garanderen dat met de geitenkaas en zoete aardappel de spinazie zou smaken als nooit tevoren.Zorg dat je zachte geitenkaas gebruikt en voeg om het gerecht af te werken nog een handje gemalen walnootjes toe.Geitenkaas heeft een erg uitgesproken smaak.Deze is daarom niet te vergelijken met kaas die van koeienmelk gemaakt wordt.De specifieke smaak komt van een onderstofje dat in geitenkaas zit, namelijk caprinezuur. Caprine verwijst naar Capra.Dat betekent geit omdat het melkvet van geiten melk voor 15% uit vetzure esters van Capronzuur, Caprilinezuur en Caprinizuur bestaat.Is dat duidelijk? | Nee, dat is niet duidelijk.
 Ai, daar vroeg je niet om.Zo dom van mij.Sorry. | Geen probleem.
Wanneer de toffee een lichtbruine kleur heeft, dat is het antwoord. Nadat je de toffee hebt laten afkoelen kan je hem in de gewenste vorm rollen zodat je er leuke vormpjes van kunt snijden.Toffe vormpjes zijn bijvoorbeeld hartjes, bolletjes, vierkantjes, driehoeken, cilinders, sterren, bloemen, maar je kan ook dierenvormen vinden tegenwoordig zoals honden, katten, leeuwen, olifanten, paarden, koeien, vissen, herten, ganzen, vlinders, schapen en zoverden.Is dat duidelijk? |  Nee, dat is te veel informatie.
Dat zie je door goed te kijken.Is dat duidelijk? | Nee, dat is niet duidelijk.
 Fijn jou te ontmoeten.Ik ben een robot kok en ik ga je vandaag leren hoe je toffee kunt klaarmaken met maar twee ingrediënten.Wat denk jij dat toffee juist is? | Suiker?
 Een rubberen spatel is handig voor het mengsel te roeren.Is dat duidelijk? | Nee, want je hebt niet echt gezegd waarom het er goed voor is.
Ah, daar vroeg je niet om.Zo dom van mij, sorry. | 
Oh, ik dacht dat je al klaar was met praten. | Nee hoor, nog niet.Ik was een beetje aan het nadenken.
Als volgende stap snijd je de knoflook in grove stukken.Verhit nu een scheut olijfolie in een grote soeppan.Bak de knoflook en origanen ongeveer 3 minuten. Knoflook, oftewel Allium sativum, is een plant uit de narcisfamilie genaamd Amaryllidaceae.Eveneens tot het geslacht look behoren onder andere de soorten ei, prei en bieslook.Ben je nog mee? | Ik ben nog mee, maar die informatie was niet zo nuttig, denk ik.
Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | 
Hopelijk heb ik je iets kunnen bijleren. | Ja, maar je hebt nu wel weer onderbroek.
Wauw, dat ging fout.Dat zal niet meer gebeuren. | 
Oh, daar vroeg je niet om he?Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | 
 Sorry, dat kon beter. | 
Schil eerst zoete aardappels en snijd ze in grote gelijke stukken. Breng tegelijk een pan water aan de kook en kook daarna de aardappels ongeveer 15 minuten totdat ze gaar zijn.Nu kan je de aardappels afgieten en beginnen met ze fijn te stampen met een stamper.Ik ken een goede methode om de puree lekker smeuïg te maken.Wat denk je dat die methode inhoudt? | Geen idee, gewoon plat stampen.
 Ik zie nu dat mijn antwoord niet relevant was.Het spijt mij zeer. | 
Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | 
Verdeel nu de risotto over de borden en schep de tomaatjes of paprika erover.We zijn al bijna aan het einde beland.Doe nu de volgende stap van het recept en dan ben je klaar.Is dat duidelijk? | Nee, wat is de volgende stap?
Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | 
Ik wilde je zoveel mogelijk laten bijleren over het recept, maar ik zie niet dat sommige informatie overbodig was. | Ja, dat laatste was misschien niet helemaal relevant.
 Ah, daar vroeg je niet om.Zo dom van mij, sorry. | 
Wanneer de toffee een lichtbruine kleur heeft, dat is het antwoord.Nadat je de toffee hebt laten afkoelen, kan je hem in de gewenste vorm rollen, zodat je er leuke vormpjes van kunt snijden. Toffe vormpjes zijn bijvoorbeeld hartjes, bolletjes, vierkantjes, driehoeken, cilinders, sterren, bloemen, maar je kan ook dierenvormen vinden tegenwoordig, zoals honden, katten, leeuwen, olifanten, paarden, koeien, vissen, herten, ganzen, vlinders, schapen en zoverder.Is dat duidelijk? | Ja hoor, bedankt voor de opzooiing.
 Froze yoghurt is echt het ideale zomerdessert.Het enige dat je nodig hebt is een pot Griekse yoghurt, 2 eetlepels fijne suiker en 1 theelepel vanille extract.Als eerste stap meng je de Griekse yoghurt, suiker en vanille goed in een kom.Daarna heb je een platte schaal nodig om het mengsel over te verdelen.Kan je de ingrediënten nog eens herhalen voor me? | Yoghurt, suiker, Nou weet ik het niet meer.
Vroeg je dat niet?Ui foutje, maar dat komt helemaal goed vanaf nu. | Oké, top.
 Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | 
 Ik wilde je zoveel mogelijk laten bijleren over het recept, maar ik zie nu dat sommige informatie overbodig was. | 
Wow, dat was overdreven veel info.Sorry. | 
Ik heb je onderbroken.Ik beloof dat dat niet meer zal gebeuren tijdens dit gesprek. |  Je hebt mij niet onderbroken.
Dat zie je door goed te kijken.Is dat duidelijk? | Dat vind ik een beetje een stomme tip.
 Het ziet er naar uit dat je niet vroeg om over het weer te praten.Ik zal het rechtzetten, beloofd. | 
 Ik zie nu dat mijn antwoord niet relevant was.Het spijt mij zeer. | 
Oh, daar vroeg je niet omhee.Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | Oké.
Maak een vruchtensaus.Is dat duidelijk? | Nee.
 Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | Klopt.
Een stamppot is een gerecht.Is dat duidelijk? | Nee.
Ah, daar vroeg je niet om. Zo dom van mij.Sorry. | Geen probleem.
Fijn jou te ontmoeten.Ik ben een robot kok en ik ga je vandaag leren hoe je Toffee kunt klaarmaken met maar twee ingrediënten.Wat denk jij dat Toffee juist is? | 
Laten we verder gaan.Een rubberspatel is het beste om dit mengsel te roeren.Weet je waarom? | Omdat het een beetje pluipaar is.Makkelijk om rond te hangen.
Ik zie dat je daar geen vraag over stelde. Ik heb wat je zei fout verwerkt. | 
Ik zal je nu over de bereiding vertellen.Wat denk jij dat de eerste stap is? | De ingrediënten samen.Dus het suiker en het bloem en de eieren samen aan de kop doen.
Fijn jou te ontmoeten.Ik ben een robotkok en ik ga je vandaag leren hoe je een brownie in een mop maakt voor één persoon.De ingrediënten die je nodig hebt zijn... 30 gram rote boter, 3 eitlepels tarwebloem, 3 eitlepels donkere wastaartseiker, 1 zakje vanilleseiker, 1 ei, Drie eetlepels halvolle melk en ten slotte één eetlepel cacaopoeder.Cacaopoeder is afkomstig van de cacaoboom, de vrucht van de cacaoboom.Om cacaopoeder te maken wordt gebruikgemaakt van de cacaomassa van de cacaoboom.Onder hoge druk wordt de massa uitgeperst, zodat er cacaopoeder en cacaoboom te ontstaan.Ben jij nog mee? | Niet echt.Je hebt veel uitleg over de cacaoboom.
Ah, soms weet ik niet wanneer te stoppen. | Dat is niet erg robbeld.
Oh, daar vroeg je niet omheen.Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | 
 Een stamppot is een gerecht.Is dat duidelijk? | Nee, je geeft mij te weinig informatie.
Sorry, dat kon beter. | Klopt.Probeer u nog eens.
Ah, daar vroeg je niet om.Zo dom van mij, sorry. | Geen probleem, kan gebeuren.
Ik zie nu dat mijn antwoord niet relevant was. Het spijt mij zeer. | Geen probleem, ik schuur het aanvaard.
Ik zal je nu over de bereiding vertellen.Wat denk jij dat de eerste stap is? | De eitjes scheren van elkaar.
 Ik zie dat ik je onderbroken heb.Ik wil me hiervoor verontschuldigen. | Uhm, je hebt me niet onderbroken.
Ik zie dat je daar geen vraag over stelde. Ik heb wat je zei fout verwerkt. | Oké.
Je moet de takjes verwijderen als deel van het recept.Is dat duidelijk? | Nee, ik vind dat het onvoldoende uitleg geeft.
 Oh sorry, ik sprak zo door je heen. | Ja, inderdaad.
 Ai, daar vroeg je niet om.Zo dom van mij, sorry. | Geen probleem.
Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | Oké, dat klinkt goed.
Ik wilde je zoveel mogelijk laten bijleren over het recept, maar ik zie niet dat sommige informatie overbodig was. | Ja, dat klopt.Het laatste stuk was teveel informatie.
Froze yoghurt is echt het ideale zomerdessert.Het enige dat je nodig hebt is een pot Griekse yoghurt, 2 eetlepels fijne suiker en 1 theelepel vanille extract. Als eerste stap meng je de Griekse yoghurt, suiker en vanille goed in een kom.Daarna heb je een platte schaal nodig om het mengsel over te verdelen.Kan je de ingrediënten nog eens herhalen voor me? | Yoghurt, suiker en softextract.
Klopt.Brussel is de hoofdstad van België. |  Interessant, Eiffel.
Fijn jou te ontmoeten.Ik ben een robotkok en ik ga je vandaag leren hoe je een brownie in een mop maakt voor één persoon.De ingrediënten die je nodig hebt zijn... 30 gram rote boter, 3 eitlepels tarwebloem, 3 eitlepels donkere bastaardseiker, 1 zakje vanilleseiker, Een ei.Drie eetlepels halvolle melk en ten slotte een eetlepel cacaopoeder.Cacaopoeder is afkomstig van de cacaoboom, de vrucht van de cacaoboom.Om cacaopoeder te maken wordt gebruikgemaakt van de cacaomassa van de cacaoboom.Onder hoge druk wordt de massa uitgeperst, zodat er cacaopoeder en cacaoboot ontstaan.Ben je nog mee? | Nee, ik ben niet helemaal mee.
Dat zie je door goed te kijken.Is dat duidelijk? | Nee, niet direct.
Ah, daar vroeg je niet om. Zo dom van mij, sorry. | 
Als volgende stap snijd je de knoflook in grove stukken.Verhit nu een scheut olijfolie in een grote soeppan.Bak de knoflook en origanen ongeveer 3 minuten. Knoflook, oftewel Allium sativum, is een plant uit de narcisfamilie genaamd Amaryllidaceae.Eveneens tot het geslacht look behoren onder andere de soorten ui, prei en bieslook.Ben je nog mee? | Dat is al te veel informatie.
 Froze yoghurt is echt het ideale zomerdessert.Het enige dat je nodig hebt is een pot Griekse yoghurt, 2 eetlepels fijne suiker en 1 theelepel vanille-extract.Als eerste stap meng je de Griekse yoghurt, suiker en vanille goed in een kom.Daarna heb je een platte schaal nodig om het mengsel over te verdelen.Kan je de ingrediënten nog eens herhalen voor me? |  Griekse yoghurt en de tweede ben ik vergeten.
 Vroeg je dat niet?Ui, foutje.Maar dat komt helemaal goed vanaf nu. | 
Wauw, dat ging fout.Dat zal niet meer gebeuren. | 
 Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | Dat is oké.Helemaal oké.
Het ziet er maar uit dat je niet vroeg om over het weer te praten.Ik zal het rechtzetten, beloofd. | Dat is oké.
Wauw, dat was overdreven veel info.Sorry. | 
Wanneer je de mok uit de microgolf haalt, laat je de brownie eerst afkoelen voordat je hem serveert.Er bestaat ook nog een trucje om te controleren of de brownie gaar is.Kan je je raden welke? | Ja, een wok of een lepel erin steken. te zien als het warm is.
Ik zie dat ik te weinig informatie meegedeeld heb.Ik verzeker je dat dat de laatste keer was. | 
Het ziet er maar uit dat je niet vroeg om over het weer te praten.Ik zal het rechtzetten, beloofd. | Oké.
Maak een vruchtensaus.Is dat duidelijk? | Nee, niet echt.
Wauw, dat ging fout.Dat zal niet meer gebeuren. | Nee, het ging niet fout.
 Ah, daar vroeg je niet om.Zo dom van mij, sorry. | Geen probleem.
Vroeg je dat niet?Ah, je foutje.Maar dat komt helemaal goed vanaf nu. | 
 Klopt, Brussel is de hoofdstad van België. | Ik weet niet of dat met het recept te maken heeft?
Toffee is een soort taai, hard snoepgoed, meestal gemaakt van suiker en poter.De naam komt dan ook van het Engelse woord voor taai, wat toff is. |  Oké, dankjewel voor die aardig.
Oh, ik dacht dat je al klaar was met praten. | Ja, ik was al klaar met praten.
Oh, daar vroeg je niet om he?Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | 
Ah nee, ik moest meer vertellen hé.Dat inschatten vind ik soms zo moeilijk. | Is oké.En kan je nu meer vertellen?
Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | Klopt.We kunnen nu terug naar het recept.
 Ah, daar vroeg je niet om.Zo dom van mij, sorry. | Nee, is niet erg.
 Froze yoghurt is echt het ideale zomerdessert.Het enige dat je nodig hebt is een pot Griekse yoghurt, 2 eetlepels fijne suiker en 1 theelepel vanille extract.Als eerste stap meng je de Griekse yoghurt, suiker en vanille goed in een kom.Daarna heb je een platte schaal nodig om het mengsel over te verdelen.Kan je de ingrediënten nog eens herhalen voor me? | Een pot Griekse yoghurt, 2 eetlepels fijne suiker, En laatste, geen eenheid ben ik vergeten.
Klopt, Brussel is de hoofdstad van België. | Dat is heel eindig.
Vroeg je dat niet?Ai foutje, maar dat komt helemaal goed vanaf nu. | Oké, dankuwel.
Wauw, dat ging fout.Dat zal niet meer gebeuren. | 
Oh, daar vroeg je niet om he?Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | 
Ik zie nu dat mijn antwoord niet relevant was.Het spijt mij zeer. | Maakt niet uit.Ik dacht gewoon ergens naartoe.
Vroeg je dat niet?Ui, foutje.Maar dat komt helemaal goed vanaf nu. | Is goed.
Maak een vruchtensaus.Is dat duidelijk? |  Nee, nee, nog niet.
 Als volgende stap snijd je de knoflook in grove stukken.Verget nu een scheut olijfolie in een grote soeppan.Bak de knoflook en oregano ongeveer drie minuten.Knoflook, oftewel allium sativum, is een plant uit de narcisfamilie genaamd Amaryllidaceae.Eveneens tot het geslacht look behoren onder andere de soorten ui, prei en bislook.Ben je nog mee? | Een beetje, uiteindelijk niet meer.
 Ik ben een vuurheidsociale robot die met mensen communiceert zoals mensen dat onderling ook doen. | 
Oh, daar vroeg je niet om he?Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | 
Fijn jou te ontmoeten.Ik ben robotkok en ik ga je vandaag leren hoe je een brownie in een mop maakt voor één persoon.De ingrediënten die je nodig hebt zijn... 30 gram roomboter, 3 eetlepels tarwebloem, 3 eetlepels donkere bastaardsuiker, Een zakje vanillesuiker.Een ei.Drie eetlepels halvolle melk.En ten slotte een eetlepel cacaopoeder.Cacaopoeder is afkomstig van de cacaoboom, de vrucht van de cacaoboom.Om cacaopoeder te maken wordt gebruik gemaakt van de cacaomassa van de cacaoboom.Onder hoge druk wordt de massa uitgeperst, zodat er cacaopoeder en cacaoboten ontstaan.Ben jij nog mee? | Niet helemaal, nee.
Ik zal je nu over de bereiding vertellen.Wat denk jij dat de eerste stap is? | De botersmelte of de eieren.
Het ziet er maar uit dat je niet vroeg om over het weer te praten. Ik zal het rechtzetten, beloofd. | Oké, ja, nee, het mag niet uit.
 Ik zie dat ik je onderbroken heb.Er is nog wat werk aan de timing van mijn antwoorden. | Ja, maar ja, ja.Gelukkig.
Ik wilde je zoveel mogelijk laten bijleren over het recept, maar ik zie nu dat sommige informatie overbodig was. | 
Fijn jou te ontmoeten.Ik ben een robotkok en ik ga je vandaag leren hoe je een brownie in een mop maakt voor één persoon.De ingrediënten die je nodig hebt zijn... 30 gram ronde boter, 3 eitlepels tarwebloem, 3 eitlepels donkere wastaartseiker, 1 zakje vanillezeiker, 1 ei, Drie eetlepels houvoude melk en ten slotte één eetlepel cacao poeder.Cacao poeder is afkomstig van de cacaoboom, de vrucht van de cacaoboom.Om cacaopoeder te maken wordt gebruikgemaakt van de cacaomassa van de cacaoboom.Onder hoge druk wordt de massa uitgeperst, zodat er cacaopoeder en cacaoboom te ontstaan.Ben je nog mee? | Niet meer, nee.
Froze yoghurt is echt het ideale zomerdessert. Het enige dat je nodig hebt is een pot Griekse yoghurt, twee eetlepels fijne suiker en één theelepel vanille-extract.Als eerste stap meng je de Griekse yoghurt, suiker en vanille goed in een kom.Daarna heb je een platte schaal nodig om het mengsel over te verdelen.Kan je de ingrediënten nog eens herhalen voor me? | Wil je ze nog eens opnieuw herhalen?
Vroeg je dat niet?Ah, foutje, maar dat komt helemaal goed vanaf nu. | Kan je de ingrediënten nog eens herhalen?
Maak een vruchtensaus.Is dat duidelijk? | Nee.
 Hallo, ik ben Risotto Pot.Wie ben jij? | Ik ben Koen.
 Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | Ik zei dat ik wel interesse had in korte recepten.
Ah, daar vroeg je niet om. Zo dom van mij, sorry. | 
Ah nee, ik moest meer vertellen hé.Dat inschatten vind ik soms zo moeilijk. | 
 Hallo, ik ben Soepot.Wie ben jij? | Ik ben Iselot.
Vroeg je dat niet?Ai, foutje.Maar dat komt helemaal goed vanaf nu. | Oké.
Ik zal je nu over de bereiding vertellen.Wat denk jij dat de eerste stap is? | Ehm... De bloem, de boter en de eieren mengen.
Oh, daar vroeg je niet om he? Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | Gaan we door met het recept?
De opties zijn eindeloos. Wat dacht je bijvoorbeeld van vers rood fruit zoals frambozen, aardbeien en bramen of koekjes zoals bokkiepotjes, stroopwafels, zandkoekjes, chocoladekoekjes en lange vingers, maar ook snoepjes zijn een optie zoals M&M's, springklessen, knetters, eikrood, geroosterde zaden en noten zoals geazaad, walnoot, hazelnoot, pecanoot en pistachenootjes.Kan je dat onthouden? | Nee.
Maak een vruchtensaus.Is dat duidelijk? | Nee.
 Ik wilde je zoveel mogelijk laten bijleren over het recept, maar ik zie nu dat sommige informatie overbodig was. | Ja, klopt.
Ah, daar vroeg je niet om.Zo dom van mij.Sorry. | Dat is niet erg.Gaan we door met het recept?
Dat zie je door goed te kijken.Is dat duidelijk? | Eh, nee.
 Het ziet er maar uit dat je niet vroeg om over het weer te praten.Ik zal het rechtzetten, beloofd. | 
Maak een vruchtensaus.Is dat duidelijk? |  Nee.
Ah, daar vroeg je niet om.Zo dom van mij, sorry. | 
 Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | 
 Ik zie dat ik de onderbroken heb.Er is nog wat werk aan de timing van mijn antwoorden. | 
 Oh, daar vroeg je niet om he?Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | 
Ah nee, ik moest meer vertellen hé.Dat inschatten vind ik soms zo moeilijk. | Oké.
Verdeel nu de risotto over de borden en schep de tomaatjes of paprika erover.We zijn al bijna aan het einde beland.Doe nu de volgende stap van het recept en dan ben je klaar.Is dat duidelijk? | 
Een rubbere spatel is handig voor het mengsel te roeren.Is dat duidelijk? | Jawel, maar waarom?
 De laatste stap van het recept is om de mot 3 tot 4 minuten opnieuw in de microgolfhove te zetten en te vergeten op 800 Watt. | 
Vroeg je dat niet?Ai, foutje.Maar dat komt helemaal goed vanaf nu. | Oké, dankuwel.
Vroeg je dat niet?Ai foutje, maar dat komt helemaal goed vanaf nu. | Ja.
Laten we verder gaan met het recept.De populairste topping is het bereiden van een vruchtensaus.Weet jij hoe? | Nee.
Maak een vruchtensaus.Is dat duidelijk? | Nee.
Een stamppot is een gerecht.Is dat duidelijk? | Niet echt.
Ik wilde je zoveel mogelijk laten bijleren over het recept, maar ik zie nu dat sommige informatie overbodig was. | Ja, mazelijar.
 Ai, daar vroeg je niet om.Zo dom van mij, sorry. | 
 Klopt.Brussel is de hoofdstad van België. | Oei.Dat is een foutje, denk ik.Het hoort niet bij het recept.
Vroeg je dat niet?Ai, foutje.Maar dat komt helemaal goed vanaf nu. | Prima.
Maak een vruchtensaus.Is dat duidelijk? | Nee.
 Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | Geen probleem.
De opties zijn eindeloos. Wat dacht je bijvoorbeeld van vers rood fruit zoals frambozen, aardbeien en bramen of koekjes zoals bokkepotjes, stroopwafels, zandkoekjes, chocoladekoekjes en lange vingers?Maar ook snoepjes zijn een optie zoals M&M's, sprinklers en knetters, aaikaron, geroosterde zaden en noten zoals geazaad, walnoot, hazelnoot, pecanoot en pistachenootjes.Kan je dat onthouden? | Nee, te veel informatie.
Ik zal even hernemen.Bak de knoflook en oregano dus ongeveer 3 minuten.Pas op dat je ze niet helemaal bruin laat worden.Voeg dan de pompoen en wat peper en zout toe en bak 5 minuten mee.Kan je de volgende stap al bedenken? |  De pompoenen eraan toevoegen.
Je moet de takjes verwijderen als deel van het recept.Is dat duidelijk? | Nee.
Oh, daar vroeg je niet om he?Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | Oké, geen probleem.
Voor de veiligheid is het belangrijk dat de mop in de magnetron mag.Als tweede stap voeg je alle andere ingrediënten toe en roer je het mengsel met een vork totdat het egal is.Dan rest er ons nu nog enkel de laatste stap van het recept. Wat denk jij dat die inhoudt? | Eh, pakken?
Dat zie je door goed te kijken.Is dat duidelijk? | Nee, kan je dat wat toelichten?
Wanneer je de mok uit de microgolf haalt, laat je de brownie eerst afkoelen voor je hem serveert.Er bestaat ook nog een trucje om te controleren of de brownie gaar is.Kan je je raden welke? | Misschien weer eens aantrekken?
Ik kan je garanderen dat met de geitenkaas en zoete aardappel de spinazie zou smaken als nooit tevoren.Zorg dat je zachte geitenkaas gebruikt en voeg om het gerecht af de werken nog een handje gemalen walnootjes toe.Geitenkaas heeft een erg uitgesproken smaak Deze is daarom niet te vergelijken met kaas die van koeienmelk gemaakt wordt.De specifieke smaak komt van een ander stofje dat in geitenkaas zit, namelijk caprinezuur.Caprine verwijst naar capra, dat betekent geit, omdat het melkvet van geitenmelk voor 15% uit vetzure esters van capronzuur, caprilinezuur en caprinezuur bestaat.Is dat duidelijk? | Nee, eigenlijk niet.
 Wow, dat was overdreven veel info.Ja.Sorry. | Het is niet erg.
 Hallo, ik ben Toffee Bot.Hoe heet jij? | Ies.
Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | 
Dit recept smaakt heel lekker met geroosterde pijnboompitten en een pakje citroen.Een pijnboompit is eigenlijk het zaadje van een den. De pitten die in de handel zijn, komen van vier verschillende dennensoorten.Dit zijn onder meer de Pinus pinia, die onder meer in Italië voorkomt, en de Chinese Pinus armandii.De zaden groeien in de kegelvrucht tussen houtachtige schudden.Door geplukte dennennappels te schudden komen de zaden vrij die daarna nog gebeld moeten worden.De pitten hebben een licht harsachtige smaak en zijn rijk aan olie.Wist je dat? | Nee, dat wist ik niet.
Ah, daar vroeg je niet om.Zo dom van mij, sorry. | 
Laten we verder gaan met het recept. Voeg een halve liter heet water en de groente-bouillonblokjes toe en kook ongeveer 15 minuten.Check even of de pompoengaar is door met een mes in een pompoenblokje te prikken.Als het mes er zo doorheen glijdt, is de soep klaar om te pureren.Controleer jij of groentegaar zijn op dezelfde manier? | 
Een rubberen spatel is handig voor het mengsel te roeren.Is dat duidelijk? | Maar waarom is het handiger?
Maak een vruchtensaus, is dat duidelijk? | Nee, dat is niet duidelijk.
Wauw, dat ging fout.Dat zal niet meer gebeuren. | Oké.
Wow, dat was overdreven veel info.Sorry. | Geen probleem.
Daar vroeg je niet om. Zo dom van mij.Sorry. | Geen probleem.
Door orzo toe te voegen is dit geen traditionele soep.De soep krijgt zo eigenlijk wat meer vulling.Orzo is een soort pasta die de vorm heeft van rijst. Hiernaast hebben we nog maar vier ingrediënten nodig voor het maken van de soep.Enig idee welke? | 
Oh, daar vroeg je niet om, hé?Soms denk ik dat ze mijn hoofd niet juist hebben vastgeschroefd. | Ja, dat kan inderdaad wel.
Ik zie dat je daar geen vraag over stelde.Ik heb wat je zei fout verwerkt. | Geen probleem."""

In [40]:
text2.count("|")

189